Fabien,

Voici les critères d’extraction et les règles de transformation à appliquer pour fabriquer nos jeux de données tests.

Comme la population AEM est faible en 2020, j’ajoute des commandes prestations plus anciennes que le 15/09 pour lesquelles il faudra que tu transformes la date.

•	ACL fictives

Extraire un jeu de données avec type presta = AEM, date de fin prestation >= 15/09/2020, et code site région =HDF ou GRE (59212, 67085)
Transformer le type prestation en ACL

Extraire un jeu de données avec type presta = AEM, date de fin prestation >= 15/06/2020 et < 15/09/2020, et code site région = HDF ou GRE (59212, 67085)
Transformer le type prestation en ACL
Ajouter 90 jours aux dates de début, dates de fin et dates d’effet résultat

Empiler les deux jeux de données

Garder sans modifier les résultats autres que VED et VEM
Transformer les résultats VED en VA2, VA3 et VAT (33 / 33 / 33 environ)
Transformer les résultats VEM en VA2, VA3, VAT et VED (25 / 25 / 25 / 25 environ)

Ainsi nous devrions avoir environ 3200 commandes EMS récentes


In [1]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
beneficiaire = dataiku.Dataset("beneficiaire")
beneficiaire_df = beneficiaire.get_dataframe()
commandepresta = dataiku.Dataset("commandepresta")
commandepresta_df = commandepresta.get_dataframe()
commandresultat = dataiku.Dataset("commandresultat")
commandresultat_df = commandresultat.get_dataframe()


# Compute recipe outputs
# TODO: Write here your actual code that computes the outputs
# NB: DSS supports several kinds of APIs for reading and writing data. Please see doc.

#recette_df = ... # Compute a Pandas dataframe to write into recette


# Write recipe outputs
#recette = dataiku.Dataset("recette")
#recette.write_with_schema(recette_df)

/applis/upr054/pur/uiuxpr054000001/code-envs/python/py_v36_administrator/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (19,41) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/applis/upr054/pur/uiuxpr054000001/code-envs/python/py_v36_administrator/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (18,40) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [3]:
beneficiaire_df.shape

(694170, 49)

# ACL fictive
•	ACL fictives

Extraire un jeu de données avec type presta = AEM, date de fin prestation >= 15/09/2020, et code site région =HDF ou GRE (59212, 67085)
Transformer le type prestation en ACL

Extraire un jeu de données avec type presta = AEM, date de fin prestation >= 15/06/2020 et < 15/09/2020, et code site région = HDF ou GRE (59212, 67085)
Transformer le type prestation en ACL
Ajouter 90 jours aux dates de début, dates de fin et dates d’effet résultat

Empiler les deux jeux de données

Garder sans modifier les résultats autres que VED et VEM
Transformer les résultats VED en VA2, VA3 et VAT (33 / 33 / 33 environ)
Transformer les résultats VEM en VA2, VA3, VAT et VED (25 / 25 / 25 / 25 environ)


In [52]:
from datetime import datetime,date, timedelta
commandepresta_df_ACL = commandepresta_df.loc[(commandepresta_df['dc_typepresta_id'].isin(['AEM'])) \
                                              & (commandepresta_df['dep'].isin([59,67,'59','67'])) &\
                                              (commandepresta_df['dd_finprevisionnelle']>=datetime(2020, 9, 15)) &\
                                             (commandepresta_df['dc_codesiteregion_id'].isin([59212, 67085,'59212','67085']))]

commandresultat_df_ACL = commandresultat_df.loc[commandresultat_df['key'].isin(commandepresta_df_ACL['key'])]


commandepresta_df_ACLbis = commandepresta_df.loc[(commandepresta_df['dc_typepresta_id'].isin(['AEM'])) \
                                                 & (commandepresta_df['dep'].isin([59,67,'59','67'])) \
                                                 &(commandepresta_df['dd_finprevisionnelle']<datetime(2020, 9, 15)) \
                                                 &(commandepresta_df['dd_finprevisionnelle']>=datetime(2020, 6, 15)) \
                                                 &(commandepresta_df['dc_codesiteregion_id'].isin([59212, 67085,'59212','67085']))]

commandresultat_df_ACLbis = commandresultat_df.loc[commandresultat_df['key'].isin(commandepresta_df_ACLbis['key'])]
commandresultat_df_ACLbis.loc[:,'dd_dateeffetresultat'] = commandresultat_df_ACLbis.loc[:,'dd_dateeffetresultat'].apply(lambda x: x+timedelta(days=90))

commandepresta_df_ACLbis.loc[:,'dd_finprevisionnelle'] = commandepresta_df_ACLbis.loc[:,'dd_finprevisionnelle'].apply(lambda x: x+timedelta(days=90))
commandepresta_df_ACLbis.loc[:,'dd_datedebut'] = commandepresta_df_ACLbis.loc[:,'dd_datedebut'].apply(lambda x: x+timedelta(days=90))

commandepresta_df_ACL = pd.concat([commandepresta_df_ACL,commandepresta_df_ACLbis],axis=0)

commandresultat_df_ACL = pd.concat([commandresultat_df_ACL,commandresultat_df_ACLbis],axis=0)

beneficiaire_df_ACL = beneficiaire_df.loc[beneficiaire_df['key'].isin(commandepresta_df_ACL['key'])]
#commandresultat_df_ACL = commandresultat_df.loc[commandresultat_df['key'].isin(commandepresta_df_ACL['key'])]

import numpy as np
nb = commandresultat_df_ACL.loc[commandresultat_df['kc_resultatsbeneficiaire_id']=='VED'].shape[0]
commandresultat_df_ACL.loc[commandresultat_df['kc_resultatsbeneficiaire_id']=='VED','kc_resultatsbeneficiaire_id']= np.random.choice(['VA2','VA3','VAT'],p=[1/3,1/3,1/3],size=nb)

nb = commandresultat_df_ACL.loc[commandresultat_df['kc_resultatsbeneficiaire_id']=='VEM'].shape[0]
commandresultat_df_ACL.loc[commandresultat_df['kc_resultatsbeneficiaire_id']=='VEM','kc_resultatsbeneficiaire_id']= np.random.choice(['VA2','VA3','VAT','VED'],p=[1/4,1/4,1/4,1/4],size=nb)


commandepresta_df_ACL.loc[:,'dc_typepresta_id'] = 'ACL'
commandresultat_df_ACL.loc[:,'dc_typepresta_id'] = 'ACL'
beneficiaire_df_ACL.loc[:,'dc_typepresta_id'] = 'ACL'

In [42]:
#commandresultat_df_ACL.head()
#commandepresta_df_ACL.head()
# VA2, VA3 et VAT 
import numpy as np
nb = commandresultat_df_ACL.loc[commandresultat_df['kc_resultatsbeneficiaire_id']=='VED'].shape[0]
commandresultat_df_ACL.loc[commandresultat_df['kc_resultatsbeneficiaire_id']=='VED','kc_resultatsbeneficiaire_id']= np.random.choice(['VA2','VA3','VAT'],p=[1/3,1/3,1/3],size=nb)

In [51]:
#VEM en VA2, VA3, VAT et VED (25 / 25 / 25 / 25 environ)
nb = commandresultat_df_ACL.loc[commandresultat_df['kc_resultatsbeneficiaire_id']=='VEM'].shape[0]
commandresultat_df_ACL.loc[commandresultat_df['kc_resultatsbeneficiaire_id']=='VEM','kc_resultatsbeneficiaire_id']= np.random.choice(['VA2','VA3','VAT','VED'],p=[1/4,1/4,1/4,1/4],size=nb)

In [49]:
#list(commandepresta_df)
#commandepresta_df['dd_finprevisionnelle']
#list(commandepresta_df_ACL['key'])

In [50]:
#commandresultat_df_ACL.head(15)

# EMS fictif

Extraire un jeu de données avec type presta = AEM, date de fin prestation >= 15/09/2020, et code site région = OCC, NAQ, NOR ou ARA (31096, 33127, 69188, 76115)
Transformer le type prestation en EMS

Extraire un jeu de données avec type presta = AEM, date de fin prestation >= 15/06/2020 et < 15/09/2020, et code site région = OCC, NAQ, NOR, ARA ou REU (31096, 33127, 69188, 76115, 97410)
Transformer le type prestation en EMS
Ajouter 90 jours aux dates de début, dates de fin et dates d’effet résultat

Empiler les deux jeux de données

Garder sans modifier les résultats autre que VED, VEM
Transformer les résultats VED et VEM en VAB et VAT (50 / 50)

Ainsi nous devrions avoir environ 3500 commandes EMS récentes


In [55]:
from datetime import datetime,date, timedelta
commandepresta_df_EMS = commandepresta_df.loc[(commandepresta_df['dc_typepresta_id'].isin(['AEM'])) & \
                                              (commandepresta_df['dd_finprevisionnelle']>=datetime(2020, 9, 15)) & \
                                              (commandepresta_df['dc_codesiteregion_id'].isin([31096, 33127, 69188, 76115,'31096', '33127', '69188','76115']))]

commandresultat_df_EMS = commandresultat_df.loc[commandresultat_df['key'].isin(commandepresta_df_EMS['key'])]


commandepresta_df_EMSbis = commandepresta_df.loc[(commandepresta_df['dc_typepresta_id'].isin(['AEM'])) &\
                                                 (commandepresta_df['dd_finprevisionnelle']<datetime(2020, 9, 15)) \
                                                 &(commandepresta_df['dd_finprevisionnelle']>=datetime(2020, 6, 15)) \
                                                 &(commandepresta_df['dc_codesiteregion_id'].isin([31096, 33127, 69188, 76115, 97410,'31096','33127','69188','76115', '97410']))]

commandresultat_df_EMSbis = commandresultat_df.loc[commandresultat_df['key'].isin(commandepresta_df_EMSbis['key'])]
commandresultat_df_EMSbis.loc[:,'dd_dateeffetresultat'] = commandresultat_df_EMSbis.loc[:,'dd_dateeffetresultat'].apply(lambda x: x+timedelta(days=90))

commandepresta_df_EMSbis.loc[:,'dd_finprevisionnelle'] = commandepresta_df_EMSbis.loc[:,'dd_finprevisionnelle'].apply(lambda x: x+timedelta(days=90))
commandepresta_df_EMSbis.loc[:,'dd_datedebut'] = commandepresta_df_EMSbis.loc[:,'dd_datedebut'].apply(lambda x: x+timedelta(days=90))

commandepresta_df_EMS = pd.concat([commandepresta_df_EMS,commandepresta_df_EMSbis],axis=0)

commandresultat_df_EMS = pd.concat([commandresultat_df_EMS,commandresultat_df_EMSbis],axis=0)

beneficiaire_df_EMS = beneficiaire_df.loc[beneficiaire_df['key'].isin(commandepresta_df_EMS['key'])]
#commandresultat_df_ACL = commandresultat_df.loc[commandresultat_df['key'].isin(commandepresta_df_ACL['key'])]

#VED et VEM en VAB et VAT (50 / 50)

import numpy as np
nb = commandresultat_df_EMS.loc[commandresultat_df['kc_resultatsbeneficiaire_id'].isin(['VED','VEM'])].shape[0]
commandresultat_df_EMS.loc[commandresultat_df['kc_resultatsbeneficiaire_id'].isin(['VED','VEM']),'kc_resultatsbeneficiaire_id']= np.random.choice(['VAB','VAT'],p=[1/2,1/2],size=nb)

#nb = commandresultat_df_ACL.loc[commandresultat_df['kc_resultatsbeneficiaire_id']=='VEM'].shape[0]
#commandresultat_df_ACL.loc[commandresultat_df['kc_resultatsbeneficiaire_id']=='VEM','kc_resultatsbeneficiaire_id']= np.random.choice(['VA2','VA3','VAT','VED'],p=[1/4,1/4,1/4,1/4],size=nb)


commandepresta_df_EMS.loc[:,'dc_typepresta_id'] = 'EMS'
commandresultat_df_EMS.loc[:,'dc_typepresta_id'] = 'EMS'
beneficiaire_df_EMS.loc[:,'dc_typepresta_id'] = 'EMS'

# •	LCR, LCN, LIR, LIN, EMS, ACL réelles

Extraire un jeu de données avec type presta = LCR, LCN, LIR, LIN, EMS, ACL avec date de fin prestation >= 01/01/2020 sur toutes les DR


In [58]:
from datetime import datetime,date, timedelta
commandepresta_df_AUTRE = commandepresta_df.loc[(commandepresta_df['dc_typepresta_id'].isin(['LCR','LCN','LIR','LIN','EMS','ACL'])) \
                                              &(commandepresta_df['dd_finprevisionnelle']>=datetime(2020, 1, 1))]

beneficiaire_df_autre = beneficiaire_df.loc[beneficiaire_df['key'].isin(commandepresta_df_AUTRE['key'])]

commandresultat_df_autre = commandresultat_df.loc[commandresultat_df['key'].isin(commandepresta_df_AUTRE['key'])]

In [59]:
beneficiaire_df_autre.shape

(386767, 49)

# •	AEM réelles

Extraire un jeu de données avec type presta = AEM avec date de fin prestation >= 01/01/2020 et <15/06/2020 sur toutes les DR

In [66]:
key_to_remove = list(commandresultat_df_EMS['key'])+list(commandresultat_df_ACL['key'])

from datetime import datetime,date, timedelta
commandepresta_df_AEM = commandepresta_df.loc[(commandepresta_df['dc_typepresta_id'].isin(['AEM'])) \
                                              &(commandepresta_df['dd_finprevisionnelle']>=datetime(2020, 1, 1)) \
                                              &(commandepresta_df['dd_finprevisionnelle']<datetime(2020, 6, 15)) \
                                             & ~(commandepresta_df['key'].isin(key_to_remove))]

beneficiaire_df_AEM = beneficiaire_df.loc[beneficiaire_df['key'].isin(commandepresta_df_AEM['key'])]

commandresultat_df_AEM = commandresultat_df.loc[commandresultat_df['key'].isin(commandepresta_df_AEM['key'])]

# Union des résultats

In [69]:
beneficiaire_glob = pd.concat([beneficiaire_df_ACL,beneficiaire_df_EMS,beneficiaire_df_autre,beneficiaire_df_AEM],axis=0)
commandepresta_glob = pd.concat([commandepresta_df_ACL,commandepresta_df_EMS,commandepresta_df_AUTRE,commandepresta_df_AEM],axis=0)
commandresultat_glob = pd.concat([commandresultat_df_ACL,commandresultat_df_EMS,commandresultat_df_autre,commandresultat_df_AEM],axis=0)

# Ecriture dans HIVE/HADOOP

In [70]:
recette = dataiku.Dataset("recette")
recette.write_with_schema(beneficiaire_glob)

recettebis = dataiku.Dataset("recettebis")
recettebis.write_with_schema(commandepresta_glob)

recettetier = dataiku.Dataset("recettetier")
recettetier.write_with_schema(commandresultat_glob)

692169 rows successfully written (YhwOv76FT6)
542976 rows successfully written (BeIB8BDNo1)
348984 rows successfully written (D5ACq2Iu8h)
